# Deflection Profile Parametric Study for El Ghadioui2020 Cantilever
El Ghadioui2020_Structural behaviour of CFRP reinforced concrete members under bending and shear loads

In [1]:
# %matplotlib widget
%matplotlib qt

In [2]:
from bmcs_cross_section.api import MKappa, TShape, EC2, BarLayer, ReinfLayer
from bmcs_beam.api import BoundaryConditions, BoundaryConfig, DeflectionProfile, BeamDesign, LoadDeflectionParamsStudy
from os.path import join
import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
import bmcs_utils.api as bu

In [3]:
# Beam B-M-C-K1 - CFRP Bars

# C 50/60
def get_dp(f_ck = None, E=None, low_kappa = None, high_kappa = None, n_kappa=None, n_load_steps=None, eps_tu=None):
    if eps_tu is None:
        eps_tu = 0.00008
    
    if n_load_steps is None:
        n_load_steps = 31
        
    if E is None:
        E = 33525
    
    if high_kappa is None:
        high_kappa = 0.00001
        
    if n_kappa is None:
        n_kappa = 200
        
    if low_kappa is None:
        low_kappa = -0.00012
        
    mc = MKappa(low_kappa = low_kappa, high_kappa = high_kappa, n_kappa = n_kappa)

#     mc.cs_design.matrix='EC2 with plateau'
    mc.cs_design.matrix_.trait_set(
            E_cc=E,
            E_ct=E,
            eps_cr=0.00006, #0.000125 * f_ck/30, # f_ctm /E,
            eps_cy=0.00175,
            eps_cu=0.0035,
            eps_tu=eps_tu,
            mu=0.0,
        f_cd = 60, # f_ck * 0.85/1.5,
        factor = 1
        )
    
    mc.cross_section_shape_.B = 400
    mc.cross_section_shape_.H = 200

    bl1 = ReinfLayer(name='carbon', z=200-35, A=4 * 70.14, matmod='carbon')
    bl1.matmod_.trait_set(E=230000, f_t=3600, factor = 1.)
    mc.cross_section_layout.add_layer(bl1)

    dp = DeflectionProfile(mc=mc, n_load_steps = n_load_steps)

    dp.beam_design.beam_conf_name = BoundaryConfig.CANTILEVER_DIST_LOAD
    dp.beam_design.L = 1000
    dp.F_scale = 1
    return dp

get_dp().interact()

In [4]:
# Test E (concrete)
E_list = np.array([30000, 35000, 40000, 45000])

dp = get_dp(n_load_steps = 100)

# following fix is just to keep the sig, and not eps the same when changing E in MULTI-LINEAR Model
E = dp.mc.cs_design.matrix_.E_cc
eps_cy = dp.mc.cs_design.matrix_.eps_cy
eps_cr= dp.mc.cs_design.matrix_.eps_cr
sig_cy = E * eps_cy
sig_cr = E * eps_cr

sig_cy_list = np.full_like(E_list, sig_cy)
sig_cr_list = np.full_like(E_list, sig_cr)

# params_config = [
#     [['E_cc', dp.mc.cs_design.matrix_, E_list], 
#      ['E_ct', dp.mc.cs_design.matrix_, E_list]],
# ]
params_config = [
    [['E_cc', dp.mc.cs_design.matrix_, E_list], 
     ['E_ct', dp.mc.cs_design.matrix_, E_list], 
     ['eps_cy', dp.mc.cs_design.matrix_, sig_cy_list / E_list], 
     ['eps_cr', dp.mc.cs_design.matrix_, sig_cr_list / E_list]],
]
LoadDeflectionParamsStudy(dp).run(params_config)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Parametric study is running...
E_cc, E_ct, eps_cy, eps_cr: (30000, 30000, -0.0019333333333333333, 6.666666666666667e-05), (35000, 35000, -0.0016571428571428572, 5.714285714285714e-05), (40000, 40000, -0.00145, 5e-05), (45000, 45000, -0.001288888888888889, 4.4444444444444447e-05), 
Parametric study finished.


In [5]:
# Test E_carbon
E_carbon = [130000, 170000, 210000, 250000]

dp = get_dp(n_kappa = 500)
E_cc = dp.mc.cs_design.matrix_.E_cc
E_list = [E_cc, E_cc + 0.01, E_cc, E_cc + 0.01] # Just to update state because changing E_carbon doesn't update the state automatically
params_config = [
    [['E', dp.mc.cross_section_layout.items[0].matmod_, E_carbon], 
     ['E_cc', dp.mc.cs_design.matrix_, E_list]]
]
LoadDeflectionParamsStudy(dp).run(params_config)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Parametric study is running...
E, E_cc: (130000, 33525.0), (170000, 33525.01), (210000, 33525.0), (250000, 33525.01), 
Parametric study finished.


In [4]:
# Test carbon strength
f_t_list = [1800, 2200, 2600, 3000]
dp = get_dp()
E_cc = dp.mc.cs_design.matrix_.E_cc
E_list = [E_cc, E_cc + 0.01, E_cc, E_cc + 0.01] # Just to update state because changing E_carbon doesn't update the state automatically
params_config = [
    [['f_t', dp.mc.cross_section_layout.items[0].matmod_, list(reversed(f_t_list))], 
     ['E_cc', dp.mc.cs_design.matrix_, E_list]]
]
LoadDeflectionParamsStudy(dp).run(params_config)

Parametric study is running...
f_t, E_cc: (3000, 33525.0), (2600, 33525.01), (2200, 33525.0), (1800, 33525.01), 
Parametric study finished.


In [8]:
# Test concrete compressive strength
f_cm_list = np.array([30, 50, 70, 90])

dp = get_dp(n_kappa=200, n_load_steps=100)

E = dp.mc.cs_design.matrix_.E_cc
E_list = np.full_like(f_cm_list, E)

params_config = [
     ['eps_cy', dp.mc.cs_design.matrix_, f_cm_list / E_list],
]
LoadDeflectionParamsStudy(dp).run(params_config)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Parametric study is running...
eps_cy: (0.0008948545861297539), (0.0014914243102162564), (0.002087994034302759), (0.0026845637583892616), 
Parametric study finished.


In [9]:
# Test concrete tensile strength
f_ctd_list = np.array([0, 2.8, 4, 4.5, 5])

dp = get_dp(eps_tu = 2*0.00008, n_load_steps=200)

E = dp.mc.cs_design.matrix_.E_cc
E_list = np.full_like(f_ctd_list, E)

params_config = [
     ['eps_cr', dp.mc.cs_design.matrix_, f_ctd_list / E_list],
]
LoadDeflectionParamsStudy(dp).run(params_config)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Parametric study is running...
eps_cr: (0.0), (8.351976137211036e-05), (0.00011931394481730052), (0.00013422818791946307), (0.00014914243102162565), 
Parametric study finished.
